<a href="https://colab.research.google.com/github/HarshiniAiyyer/ESGInvest/blob/main/Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Initial Setup

In [1]:
!pip install -q opendatasets deap

In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/pritish509/s-and-p-500-esg-risk-ratings')

Skipping, found downloaded files in "./s-and-p-500-esg-risk-ratings" (use force=True to force download)


username : harshiniaiyyer pwd: db9694435b6d0b386a9c18289bb0f443

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from deap import base, creator, tools, algorithms
import random

In [4]:
df = pd.read_csv('/content/s-and-p-500-esg-risk-ratings/SP 500 ESG Risk Ratings.csv')
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df.head()

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
0,EMN,Eastman Chemical Company,"200 South Wilcox Drive\nKingsport, TN 37662\nU...",Basic Materials,Specialty Chemicals,"14,000",Eastman Chemical Company operates as a special...,25.3,12.8,6.6,5.8,Moderate Controversy Level,2.0,50th percentile,Medium
1,DPZ,Domino's Pizza Inc.,"30 Frank Lloyd Wright Drive\nAnn Arbor, MI 481...",Consumer Cyclical,Restaurants,"6,500","Domino's Pizza, Inc., through its subsidiaries...",29.2,10.6,6.3,12.2,Moderate Controversy Level,2.0,66th percentile,Medium
2,DVA,Davita Inc.,"2000 16th Street\nDenver, CO 80202\nUnited States",Healthcare,Medical Care Facilities,"70,000",DaVita Inc. provides kidney dialysis services ...,22.6,0.1,8.4,14.1,Moderate Controversy Level,2.0,38th percentile,Medium
3,DRI,"Darden Restaurants, Inc.","1000 Darden Center Drive\nOrlando, FL 32837\nU...",Consumer Cyclical,Restaurants,"187,384","Darden Restaurants, Inc., together with its su...",27.5,7.9,4.6,15.0,Moderate Controversy Level,2.0,59th percentile,Medium
4,ZTS,Zoetis Inc.,"10 Sylvan Way\nParsippany, NJ 07054\nUnited St...",Healthcare,Drug Manufacturers - Specialty & Generic,"14,100","Zoetis Inc. engages in the discovery, developm...",18.8,3.2,8.7,6.8,Moderate Controversy Level,2.0,23rd percentile,Low


In [5]:
# Drop conflicting columns if necessary
df = df[['Symbol', 'Name','Sector','Total ESG Risk score']]

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Initialize scaler
sc = MinMaxScaler()

# Normalize ESG Risk score
df['Normalized ESG Risk Score'] = sc.fit_transform(df[['Total ESG Risk score']])

<ipython-input-6-552623327707>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Normalized ESG Risk Score'] = sc.fit_transform(df[['Total ESG Risk score']])


In [7]:
import numpy as np

# Define population size and subset size
population_size = 10
subset_size = 5

# Initialize population with random subsets
def initialize_population(df, population_size, subset_size):
    population = []
    for _ in range(population_size):
        subset = np.random.choice(df.index, subset_size, replace=False)
        population.append(subset)
    return np.array(population)

population = initialize_population(df, population_size, subset_size)

In [8]:
def fitness_function(subset, df):
    subset_df = df.iloc[subset]
    return np.mean(subset_df['Normalized ESG Risk Score'])

In [9]:
def select(population, fitness_scores, num_parents):
    # Select indices of the best-performing subsets
    parents_indices = np.argsort(fitness_scores)[-num_parents:]
    return population[parents_indices]

In [10]:
def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

def mutate(subset, df, mutation_rate=0.1):
    if np.random.rand() < mutation_rate:
        idx = np.random.randint(len(subset))
        new_idx = np.random.choice(df.index)
        subset[idx] = new_idx
    return subset

In [11]:
num_generations = 20
num_parents = 6

for generation in range(num_generations):
    fitness_scores = np.array([fitness_function(subset, df) for subset in population])

    # Select parents
    parents = select(population, fitness_scores, num_parents)

    # Generate new population
    new_population = []
    for i in range(0, num_parents, 2):
        parent1, parent2 = parents[i], parents[i+1]
        child1, child2 = crossover(parent1, parent2)
        new_population.append(mutate(child1, df))
        new_population.append(mutate(child2, df))

    population = np.array(new_population)

In [13]:
# Final evaluation
final_fitness_scores = np.array([fitness_function(subset, df) for subset in population])
best_subset = population[np.argmax(final_fitness_scores)]

# Get recommended companies
recommended_companies = df.iloc[best_subset]
print("Recommended Companies:")
print(recommended_companies[['Symbol', 'Name','Sector','Total ESG Risk score']])

Recommended Companies:
    Symbol                            Name              Sector  \
399    MMM                      3m Company         Industrials   
367    ADM  Archer Daniels Midland Company  Consumer Defensive   
345     BA                  Boeing Company         Industrials   
351  BRK-B      Berkshire Hathaway Class B  Financial Services   
285   DXCM                    Dexcom, Inc.          Healthcare   

     Total ESG Risk score  
399                  37.3  
367                  31.8  
345                  39.6  
351                  20.7  
285                  24.0  
